# Necessary Imports Needed to Run Model

In [ ]:
import zipfile
import os
import random
import copy
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchvision import models
from sklearn.utils import shuffle
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager
from collections import OrderedDict

#XCeption:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.xception import Xception


# Mount Google Drive and Extract Necessary Folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# define which percent data you want to use
data_folder = '100percent'

In [ ]:
# Unzipping the Food-101 dataset - training
zip_file_path = '/content/drive/MyDrive/EC523-Project/data/zip/food101_' + data_folder + '.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/train')

zip_file_path = '/content/drive/MyDrive/EC523-Project/data/zip/food101_test.zip'
# Unzipping the Food-101 dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/train')

classes = sorted(os.listdir('/content/train/food101_' + data_folder + '/train'))
print(len(classes))

101


In [ ]:
train_images_path = '/content/train/food101_' + data_folder + '/train'
test_images_path = '/content/train/food101_test'

# Lists to store training and testing data paths
train_data = []
test_data = []


for food_category in os.listdir(train_images_path):
    category_path = os.path.join(train_images_path, food_category)
    images = os.listdir(category_path)
    train_images = images[:]

    # Add image paths to the train and test lists
    train_data.extend([f"{food_category}/{img}" for img in train_images])

for food_category in os.listdir(test_images_path):
    category_path = os.path.join(test_images_path, food_category)
    images = os.listdir(category_path)
    test_images = images[:]
    # Add image paths to the train and test lists
    test_data.extend([f"{food_category}/{img}" for img in test_images])

# Create a Meta Folder to Emulate Pytorch Dataset

In [ ]:
directory_path = '/content/train/meta'

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")

# File path for the file to create within the directory
file_path = os.path.join(directory_path, 'train.txt')
file_path = os.path.join(directory_path, 'test.txt')
file_path = os.path.join(directory_path, 'classes.txt')


# Write the train and test data to files
with open('/content/train/meta/train.txt', 'w') as f:
    for item in train_data:
        f.write("%s\n" % item)

with open('/content/train/meta/test.txt', 'w') as f:
    for item in test_data:
        f.write("%s\n" % item)

with open('/content/train/meta/classes.txt', 'w') as f:
    for item in classes:
        f.write("%s\n" % item)

# Display the first 5 lines of each file as a sample
print("Testing images")
print("\n".join(test_data[:5]))
print("\nTraining images")
print("\n".join(train_data[:5]))

In [ ]:
#Check Meta Folder
os.listdir('/content/train/meta')

# Dataframe Prep and Data Augmentation

In [ ]:
train_df = pd.read_csv('/content/train/meta/train.txt', header = None, names=['path'])
train_df.head()

In [ ]:
test_df = pd.read_csv('/content/train/meta/test.txt', header = None, names=['path'])
test_df.head()

In [ ]:
def spliter(data, class_or_id='id'):
    if class_or_id.upper() == 'CLASS':
        output = data.split('/')[0]

    else:
        output = data.split('/')[-1]
    return output

In [ ]:
train_df['label'] = train_df['path'].map(lambda x: spliter(data = x, class_or_id = 'Class'))
train_df.head()

In [ ]:
train_df['idx'] = train_df['path'].map(lambda x: spliter(x))
train_df.head()

In [ ]:
test_df['label'] = test_df['path'].map(lambda x: spliter(x, 'class'))
test_df['idx'] = test_df['path'].map(lambda x: spliter(x))
test_df.head()

In [ ]:
mini_101 = train_df['label'].unique()[:101]
mini_101 = [f.upper() for f in mini_101]
mini_101

In [ ]:
def prepare_data(label):
    if label.upper() in mini_101:
        return label
    else:
        return 'Others'

In [ ]:
train_df['label'] = train_df['label'].map(lambda x: prepare_data(x))
test_df['label'] = test_df['label'].map(lambda x: prepare_data(x))

In [ ]:
train_df['label'].unique()

In [ ]:
def add_exact_path_train(path):
    return '/content/train/food101_' + data_folder + '/train/'+path

def add_exact_path_test(path):
    return '/content/train/food101_test/'+path

In [ ]:
train_df['path'] = train_df[['path']].apply(add_exact_path_train, axis=1)
test_df['path'] = test_df[['path']].apply(add_exact_path_test, axis=1)
train_df.head()

In [ ]:
from matplotlib import pyplot as plt
img = plt.imread(train_df['path'].iloc[0])
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = datagen.flow_from_dataframe(dataframe = train_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32, shuffle=True, validate_filenames=False)

In [ ]:
test_gen = datagen.flow_from_dataframe(dataframe = test_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32, shuffle=True, validate_filenames=False)

# Model Training

In [ ]:
Xception = Xception(weights='imagenet', include_top=False)
x = Xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
# x = Dropout(0.4)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(101, activation='softmax')(x)

model = Model(inputs=Xception.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen,
                              steps_per_epoch = len(train_gen) // 32,
                    epochs=300,
                    verbose=1)


# Model Evaluation

In [ ]:
result = model.evaluate(test_gen, batch_size=32)
print(result)

In [ ]:
# Accuracy plot
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Training Accuracy'], loc='upper left')
plt.show()